# Food Recommendation Model Overview

This code implements a neural network model using PyTorch for recommending food items based on user-specific inputs such as age, weight, and activity level. The model processes various features like the physical dimensions of food images (width, height) and user characteristics to predict the best-suited food class from a predefined set of 12 food items. The main components of the code are as follows:

### Data Preparation

1. **Food Classes**: 
    - A list of 12 food items such as "nasi", "ayam", "ikan", etc., each associated with nutritional information (calories, protein, fat).

2. **Nutritional Values**:
    - Dummy nutritional values for each food class are provided, defining calories, protein, and fat per 100 grams of food.

3. **User Data**:
    - User-specific attributes such as age, weight, and activity levels are defined as input features to the model.

4. **Dataset Generation**:
    - A dummy dataset is created with random values for food dimensions, user features, and calculated nutritional values for training the model.

### Model Architecture

The `FoodRecommendationModel` is a feedforward neural network built with the following layers:
- **Input Layer**: The input size is set to 7, representing the features: food image dimensions (width, height), user-specific data (age, weight, activity level), and the nutritional information (calories, protein, fat).
- **Two Hidden Layers**: Each hidden layer contains 128 neurons with ReLU activation functions.
- **Output Layer**: This layer produces a probability distribution across the 12 food classes, indicating the recommended food based on the input.

### Model Training

- **Loss Function**: The model uses `CrossEntropyLoss`, suitable for multi-class classification tasks.
- **Optimizer**: Adam optimizer with a learning rate of 0.001 is used to update the model's parameters during training.
- **Training Loop**: A dummy training loop is run for 10 epochs with random data to simulate the training process.

### Example Prediction

After training, the model can predict the recommended food class based on the given input features. For instance, providing inputs such as:
- Food dimensions: `250 x 300` 
- Food weight: `150g`
- User details: age `36`, weight `75kg`, and activity level `1.75`
The model outputs a recommendation for the most suitable food from the available classes.



In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np

# Dummy data food classes
food_classes = ["nasi", "ayam", "kentang", "ikan", "sayur", "buah", "telur", "daging", "susu", "roti", "keju", "mie"]
num_classes = len(food_classes)

# Dummy nutritional values for each food class
nutritional_values = {
    "nasi": {"kalori": 130, "protein": 2.5, "lemak": 0.3},
    "ayam": {"kalori": 239, "protein": 27, "lemak": 14},
    "kentang": {"kalori": 77, "protein": 2, "lemak": 0.1},
    "ikan": {"kalori": 206, "protein": 22, "lemak": 12},
    "sayur": {"kalori": 25, "protein": 1.8, "lemak": 0.2},
    "buah": {"kalori": 52, "protein": 0.5, "lemak": 0.2},
    "telur": {"kalori": 155, "protein": 13, "lemak": 11},
    "daging": {"kalori": 250, "protein": 26, "lemak": 15},
    "susu": {"kalori": 42, "protein": 3.4, "lemak": 1},
    "roti": {"kalori": 265, "protein": 9, "lemak": 3.2},
    "keju": {"kalori": 402, "protein": 25, "lemak": 33},
    "mie": {"kalori": 138, "protein": 4.5, "lemak": 2}
}

# Dummy user data
user_data = {
    "user_age": [25, 30, 36, 40, 50],
    "user_weight": [70, 80, 90, 60, 75],
    "activity_level": [1.2, 1.5, 1.75, 2.0]  # This represents activity multipliers (sedentary, moderate, active, very active)
}

# Creating dummy dataset for training
def create_dummy_dataset(num_samples=100):
    data = []
    for _ in range(num_samples):
        width = np.random.randint(50, 500)  # Dummy width
        height = np.random.randint(50, 500)  # Dummy height
        food_class_idx = np.random.randint(0, num_classes)  # Random food class
        food_class = food_classes[food_class_idx]
        food_weight = np.random.uniform(50, 300)  # Random food weight
        user_age = np.random.choice(user_data["user_age"])
        user_weight = np.random.choice(user_data["user_weight"])
        activity_level = np.random.choice(user_data["activity_level"])

        # Get nutritional data based on the food class and weight
        nutrisi = nutritional_values[food_class]
        kalori = (nutrisi["kalori"] / 100) * food_weight
        protein = (nutrisi["protein"] / 100) * food_weight
        lemak = (nutrisi["lemak"] / 100) * food_weight

        # Store the data
        data.append([width, height, food_class_idx, food_weight, user_age, user_weight, activity_level, kalori, protein, lemak])

    columns = ['width', 'height', 'food_class_idx', 'food_weight', 'user_age', 'user_weight', 'activity_level', 'kalori', 'protein', 'lemak']
    return pd.DataFrame(data, columns=columns)

# Generate dataset
dataset = create_dummy_dataset(num_samples=500)
print(dataset.head())

# Save dataset for further use
dataset.to_csv('food_recommendation_dummy_data.csv', index=False)

   width  height  food_class_idx  food_weight  user_age  user_weight  \
0    308     478               9   157.641453        25           70   
1    499      57               7   275.704143        50           70   
2    422     222               9    55.604829        30           60   
3    289      79               6    74.685336        30           80   
4    452     147               8   216.988114        25           60   

   activity_level      kalori    protein      lemak  
0             1.5  417.749851  14.187731   5.044527  
1             1.5  689.260356  71.683077  41.355621  
2             1.2  147.352798   5.004435   1.779355  
3             2.0  115.762272   9.709094   8.215387  
4             2.0   91.135008   7.377596   2.169881  


In [4]:
class FoodRecommendationModel(nn.Module):
    def __init__(self, num_classes, input_size=7, hidden_size=128):
        super(FoodRecommendationModel, self).__init__()
        # Input size corresponds to width, height, user features, etc.
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, num_classes)  # Output for food classes (rekomendasi makanan)
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)  # No activation since it's multi-class prediction
        return x

# Hyperparameters
input_size = 7  # Features: width, height, user_age, user_weight, activity_level, etc.
hidden_size = 128
num_classes = len(food_classes)

# Initialize model
model = FoodRecommendationModel(num_classes=num_classes, input_size=input_size)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()  # Multi-class classification task
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Dummy training loop (without actual data loader for now)
num_epochs = 10
for epoch in range(num_epochs):
    # Dummy input, let's assume we have a batch of size 64
    inputs = torch.randn(64, input_size)
    labels = torch.randint(0, num_classes, (64,))  # Random food class labels
    
    # Forward pass
    outputs = model(inputs)
    loss = criterion(outputs, labels)
    
    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch+1) % 2 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [2/10], Loss: 2.4693
Epoch [4/10], Loss: 2.4870
Epoch [6/10], Loss: 2.4898
Epoch [8/10], Loss: 2.4986
Epoch [10/10], Loss: 2.4790


In [5]:
# Example prediction with all 7 features
user_input = torch.tensor([[250, 300, 5, 150, 36, 75, 1.75]])  # Input: width, height, food_class_idx, food_weight, user_age, user_weight, activity_level
output = model(user_input)
predicted_food_class = torch.argmax(output, dim=1)
print(f"Rekomendasi makanan: {food_classes[predicted_food_class.item()]}")


Rekomendasi makanan: buah


## Improve 1.2

### Personalized Nutritional Goals

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np

# Dummy data food classes
food_classes = ["nasi", "ayam", "kentang", "ikan", "sayur", "buah", "telur", "daging", "susu", "roti", "keju", "mie"]
num_classes = len(food_classes)

# Dummy nutritional values for each food class
nutritional_values = {
    "nasi": {"kalori": 130, "protein": 2.5, "lemak": 0.3},
    "ayam": {"kalori": 239, "protein": 27, "lemak": 14},
    "kentang": {"kalori": 77, "protein": 2, "lemak": 0.1},
    "ikan": {"kalori": 206, "protein": 22, "lemak": 12},
    "sayur": {"kalori": 25, "protein": 1.8, "lemak": 0.2},
    "buah": {"kalori": 52, "protein": 0.5, "lemak": 0.2},
    "telur": {"kalori": 155, "protein": 13, "lemak": 11},
    "daging": {"kalori": 250, "protein": 26, "lemak": 15},
    "susu": {"kalori": 42, "protein": 3.4, "lemak": 1},
    "roti": {"kalori": 265, "protein": 9, "lemak": 3.2},
    "keju": {"kalori": 402, "protein": 25, "lemak": 33},
    "mie": {"kalori": 138, "protein": 4.5, "lemak": 2}
}

# Dummy user data
user_data = {
    "user_age": [25, 30, 36, 40, 50],
    "user_weight": [70, 80, 90, 60, 75],
    "user_height": [170, 165, 180, 155, 175],  # Tinggi dalam cm
    "user_gender": ["M", "F"],  # M untuk pria, F untuk wanita
    "activity_level": [1.2, 1.5, 1.75, 2.0]  # Faktor aktivitas
}

# Menghitung BMR berdasarkan rumus Harris-Benedict
def calculate_bmr(age, weight, height, gender):
    if gender == "M":
        bmr = 88.362 + (13.397 * weight) + (4.799 * height) - (5.677 * age)
    else:
        bmr = 447.593 + (9.247 * weight) + (3.098 * height) - (4.330 * age)
    return bmr

# Menghitung TDEE
def calculate_tdee(bmr, activity_level):
    return bmr * activity_level

# Membuat dataset dummy dengan TDEE dan makronutrien
def create_dummy_dataset_with_tdee(num_samples=100):
    data = []
    for _ in range(num_samples):
        user_age = np.random.choice(user_data["user_age"])
        user_weight = np.random.choice(user_data["user_weight"])
        user_height = np.random.choice(user_data["user_height"])
        user_gender = np.random.choice(user_data["user_gender"])
        activity_level = np.random.choice(user_data["activity_level"])

        # Hitung BMR dan TDEE
        bmr = calculate_bmr(user_age, user_weight, user_height, user_gender)
        tdee = calculate_tdee(bmr, activity_level)

        # Dummy values for food
        width = np.random.randint(50, 500)
        height = np.random.randint(50, 500)
        food_class_idx = np.random.randint(0, num_classes)
        food_weight = np.random.uniform(50, 300)

        nutrisi = nutritional_values[food_classes[food_class_idx]]
        kalori = (nutrisi["kalori"] / 100) * food_weight
        protein = (nutrisi["protein"] / 100) * food_weight
        lemak = (nutrisi["lemak"] / 100) * food_weight

        data.append([width, height, food_class_idx, food_weight, user_age, user_weight, user_height, user_gender, activity_level, bmr, tdee, kalori, protein, lemak])

    columns = ['width', 'height', 'food_class_idx', 'food_weight', 'user_age', 'user_weight', 'user_height', 'user_gender', 'activity_level', 'bmr', 'tdee', 'kalori', 'protein', 'lemak']
    return pd.DataFrame(data, columns=columns)

# Generate dataset
dataset = create_dummy_dataset_with_tdee(num_samples=500)
print(dataset.head())

# Save dataset for further use
dataset.to_csv('food_recommendation_dummy_data_with_tdee.csv', index=False)

# Define the model class for food recommendation
class FoodRecommendationModelWithTDEE(nn.Module):
    def __init__(self, num_classes, input_size=9, hidden_size=128):
        super(FoodRecommendationModelWithTDEE, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)  # No activation since it's multi-class prediction
        return x

# Hyperparameters
input_size = 9  # Features: width, height, user features, tdee, etc.
hidden_size = 128
num_classes = len(food_classes)

# Initialize model
model = FoodRecommendationModelWithTDEE(num_classes=num_classes, input_size=input_size)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()  # Multi-class classification task
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Dummy training loop (without actual data loader for now)
num_epochs = 10
for epoch in range(num_epochs):
    # Dummy input, let's assume we have a batch of size 64
    inputs = torch.randn(64, input_size)
    labels = torch.randint(0, num_classes, (64,))  # Random food class labels
    
    # Forward pass
    outputs = model(inputs)
    loss = criterion(outputs, labels)
    
    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch+1) % 2 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Example prediction with all 9 features
user_input = torch.tensor([[250, 300, 5, 150, 36, 75, 170, 1, 1.75]])  # Input: width, height, food_class_idx, food_weight, user_age, user_weight, user_height, user_gender, activity_level
output = model(user_input)
predicted_food_class = torch.argmax(output, dim=1)
print(f"Rekomendasi makanan: {food_classes[predicted_food_class.item()]}")


   width  height  food_class_idx  food_weight  user_age  user_weight  \
0    114     231               4    80.039184        36           90   
1    222     136               0   140.363610        40           60   
2    260     179               6    86.081333        36           75   
3    237      57               0   174.999437        40           60   
4    285     395               0   182.985531        30           70   

   user_height user_gender  activity_level       bmr        tdee      kalori  \
0          175           M            1.20  1929.545  2315.45400   20.009796   
1          175           M            1.75  1504.927  2633.62225  182.472693   
2          155           M            2.00  1632.610  3265.22000  133.426067   
3          180           F            1.50  1386.853  2080.27950  227.499268   
4          170           F            1.20  1491.643  1789.97160  237.881191   

     protein     lemak  
0   1.440705  0.160078  
1   3.509090  0.421091  
2  11.19057

# Improve 1.3

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np

# Dummy data food classes
food_classes = ["nasi", "ayam", "kentang", "ikan", "sayur", "buah", "telur", "daging", "susu", "roti", "keju", "mie"]
num_classes = len(food_classes)

# Dummy nutritional values for each food class
nutritional_values = {
    "nasi": {"kalori": 130, "protein": 2.5, "lemak": 0.3, "karbohidrat": 28},
    "ayam": {"kalori": 239, "protein": 27, "lemak": 14, "karbohidrat": 0},
    "kentang": {"kalori": 77, "protein": 2, "lemak": 0.1, "karbohidrat": 17},
    "ikan": {"kalori": 206, "protein": 22, "lemak": 12, "karbohidrat": 0},
    "sayur": {"kalori": 25, "protein": 1.8, "lemak": 0.2, "karbohidrat": 5},
    "buah": {"kalori": 52, "protein": 0.5, "lemak": 0.2, "karbohidrat": 13},
    "telur": {"kalori": 155, "protein": 13, "lemak": 11, "karbohidrat": 1.1},
    "daging": {"kalori": 250, "protein": 26, "lemak": 15, "karbohidrat": 0},
    "susu": {"kalori": 42, "protein": 3.4, "lemak": 1, "karbohidrat": 5},
    "roti": {"kalori": 265, "protein": 9, "lemak": 3.2, "karbohidrat": 49},
    "keju": {"kalori": 402, "protein": 25, "lemak": 33, "karbohidrat": 1.3},
    "mie": {"kalori": 138, "protein": 4.5, "lemak": 2, "karbohidrat": 25}
}

# Fitur baru: Memori Input Harian
class DailyIntakeMemory:
    def __init__(self, target_calories, target_protein, target_fat, target_carbs):
        self.target_calories = target_calories
        self.target_protein = target_protein
        self.target_fat = target_fat
        self.target_carbs = target_carbs
        self.consumed_calories = 0
        self.consumed_protein = 0
        self.consumed_fat = 0
        self.consumed_carbs = 0

    def add_food(self, food_name, food_weight):
        if food_name in nutritional_values:
            nutrisi = nutritional_values[food_name]
            self.consumed_calories += (nutrisi["kalori"] / 100) * food_weight
            self.consumed_protein += (nutrisi["protein"] / 100) * food_weight
            self.consumed_fat += (nutrisi["lemak"] / 100) * food_weight
            self.consumed_carbs += (nutrisi["karbohidrat"] / 100) * food_weight

    def remaining_intake(self):
        return {
            "remaining_calories": self.target_calories - self.consumed_calories,
            "remaining_protein": self.target_protein - self.consumed_protein,
            "remaining_fat": self.target_fat - self.consumed_fat,
            "remaining_carbs": self.target_carbs - self.consumed_carbs
        }

# Mekanisme Feedback: Sesuaikan rekomendasi makan berikutnya berdasarkan riwayat harian
def adjust_recommendations(memory):
    remaining = memory.remaining_intake()
    print("Kebutuhan sisa untuk hari ini:")
    print(f"Kalori tersisa: {remaining['remaining_calories']:.2f} kcal")
    print(f"Protein tersisa: {remaining['remaining_protein']:.2f} gram")
    print(f"Lemak tersisa: {remaining['remaining_fat']:.2f} gram")
    print(f"Karbohidrat tersisa: {remaining['remaining_carbs']:.2f} gram")
    print("\nRekomendasi makanan berikutnya sesuai kebutuhan:")
    
    # Menyusun rekomendasi berdasarkan kebutuhan yang tersisa
    recommended_foods = []
    for food, nutrisi in nutritional_values.items():
        if (nutrisi["kalori"] <= remaining["remaining_calories"] and 
            nutrisi["protein"] <= remaining["remaining_protein"] and 
            nutrisi["lemak"] <= remaining["remaining_fat"] and 
            nutrisi["karbohidrat"] <= remaining["remaining_carbs"]):
            recommended_foods.append(food)

    if recommended_foods:
        print("Makanan yang direkomendasikan:", recommended_foods)
    else:
        print("Tidak ada makanan yang sepenuhnya memenuhi kriteria. Coba makanan dengan nilai gizi terdekat.")
    
    return recommended_foods

# Contoh penggunaan fitur memori harian dan penyesuaian rekomendasi
# Target harian contoh: 2000 kalori, 150g protein, 70g lemak, 250g karbohidrat
daily_memory = DailyIntakeMemory(target_calories=2000, target_protein=150, target_fat=70, target_carbs=250)

# Pengguna memasukkan makanan untuk sarapan
print("\nInput sarapan: Ayam 200g")
daily_memory.add_food("ayam", 200)

# Tampilkan kebutuhan yang tersisa setelah sarapan
adjust_recommendations(daily_memory)

# Pengguna memasukkan makanan untuk makan siang
print("\nInput makan siang: Roti 100g")
daily_memory.add_food("roti", 100)

# Tampilkan kembali kebutuhan yang tersisa setelah makan siang
adjust_recommendations(daily_memory)

# Model pelatihan yang digunakan sama seperti sebelumnya
class FoodRecommendationModelWithTDEE(nn.Module):
    def __init__(self, num_classes, input_size=9, hidden_size=128):
        super(FoodRecommendationModelWithTDEE, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)  # No activation since it's multi-class prediction
        return x

# Hyperparameters
input_size = 9  # Features: width, height, user features, tdee, etc.
hidden_size = 128
num_classes = len(food_classes)

# Initialize model
model = FoodRecommendationModelWithTDEE(num_classes=num_classes, input_size=input_size)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()  # Multi-class classification task
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Dummy training loop (without actual data loader for now)
num_epochs = 10
for epoch in range(num_epochs):
    # Dummy input, let's assume we have a batch of size 64
    inputs = torch.randn(64, input_size)
    labels = torch.randint(0, num_classes, (64,))  # Random food class labels
    
    # Forward pass
    outputs = model(inputs)
    loss = criterion(outputs, labels)
    
    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch+1) % 2 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Contoh prediksi makanan dengan model
user_input = torch.tensor([[250, 300, 5, 150, 36, 75, 170, 1, 1.75]])  # Input: width, height, food_class_idx, food_weight, user_age, user_weight, user_height, user_gender, activity_level
output = model(user_input)
predicted_food_class = torch.argmax(output, dim=1)
print(f"Rekomendasi makanan: {food_classes[predicted_food_class.item()]}")



Input sarapan: Ayam 200g
Kebutuhan sisa untuk hari ini:
Kalori tersisa: 1522.00 kcal
Protein tersisa: 96.00 gram
Lemak tersisa: 42.00 gram
Karbohidrat tersisa: 250.00 gram

Rekomendasi makanan berikutnya sesuai kebutuhan:
Makanan yang direkomendasikan: ['nasi', 'ayam', 'kentang', 'ikan', 'sayur', 'buah', 'telur', 'daging', 'susu', 'roti', 'keju', 'mie']

Input makan siang: Roti 100g
Kebutuhan sisa untuk hari ini:
Kalori tersisa: 1257.00 kcal
Protein tersisa: 87.00 gram
Lemak tersisa: 38.80 gram
Karbohidrat tersisa: 201.00 gram

Rekomendasi makanan berikutnya sesuai kebutuhan:
Makanan yang direkomendasikan: ['nasi', 'ayam', 'kentang', 'ikan', 'sayur', 'buah', 'telur', 'daging', 'susu', 'roti', 'keju', 'mie']
Epoch [2/10], Loss: 2.4847
Epoch [4/10], Loss: 2.4672
Epoch [6/10], Loss: 2.4993
Epoch [8/10], Loss: 2.5092
Epoch [10/10], Loss: 2.4905
Rekomendasi makanan: sayur
